In [18]:
import pandas as pd
from yelpapi import YelpAPI

# 1. Process columns


In [4]:
df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/data/stockholm_housing_df_RAW.csv")

df['fee'] = df['fee'].str.rstrip(' kr/mån')

#Drop 'location' column, as it is redundant
df = df.drop(columns=["location"])

#Split size of apartment and number of rooms into separate columns
df[['size', 'rooms']] = df['size:rooms'].str.split("  ", 1, expand=True)
df = df.drop(columns=["size:rooms"])
#Clean up the columns size and room, removing m2 and "rum"
df['size'] = df['size'].str.rstrip('m²')
df["rooms"] = df["rooms"].str.rstrip(' rum')

#Split adress and floor of apt into separate columns
df[['adress', 'floor']] = df['adress'].str.split(",", 1, expand=True)

#Clean up sold_date, sale_price, value_dev, ppsqm
df["sold_date"] = df["sold_date"].str.lstrip('såld')
df["sale_price"] = df["sale_price"].str.lstrip('slutpris ')
df["sale_price"] = df["sale_price"].str.rstrip(' kr')
df["ppsqm"] = df["ppsqm"].str.rstrip(" kr/m²")
df["value_dev"] = df["value_dev"].str.rstrip(" %")
df["value_dev"] = df["value_dev"].str.lstrip("+")

# Introduce dummy variables in the following way: Balkong = 1, Hiss = 2, Balkong&Hiss = 3, Uteplats = 4
df["features"] = df.features.map( {'balkong':1 , 'hiss':2, 'balkong&hiss':3, 'uteplats': 4, 'NaN':5} )

#Remove everything except numerical values from floor
df['floor'] = df['floor'].str.extract('(\d+)', expand=False)

#Bug where value_dev ends up in wrong columns, due to old posting
bad_index = df[df["ppsqm"].str.find("%") != -1].index
df = df.drop(bad_index, axis=0)

column_list = ["ppsqm", "size", "rooms", "sale_price","fee"]

def fix_numeric(column_list, df):
    new_df = df.copy()
    for i in column_list: 
        new_df[i] = new_df[i].str.replace(" ", "")
        new_df[i] = pd.to_numeric(new_df[i] , errors='coerce')
        new_df = new_df.dropna(subset=[i])
        new_df[i] = new_df[i].astype('int')
    return new_df

cleaned_housing_df = fix_numeric(column_list,df.copy() )
#Reset index after removing rows
cleaned_housing_df = cleaned_housing_df.reset_index(drop=True)
#cleaned_housing_df.to_csv("stockholm_housing_df_CLEANED.csv",index=False)


In [5]:
cleaned_housing_df

,adress,fee,features,sale_price,sold_date,value_dev,ppsqm,district,size,rooms,floor
0,frejgatan 50,2391,1.0,6450000,15 januari 2022,NaN,153571,vasastan,42,2,None
1,rådmansgatan 86,2021,4.0,7200000,15 januari 2022,13,122034,vasastan,59,2,None
2,sankt eriksgatan 109,436,2.0,3150000,14 januari 2022,19,126000,vasastan,25,1,None
3,torsgatan 58,1733,2.0,3435000,14 januari 2022,15,110806,vasastan,31,1,1
4,torsplan 8,3950,3.0,10000000,14 januari 2022,11,153846,vasastan,65,3,9
...,...,...,...,...,...,...,...,...,...,...,...
10471,kammakargatan 70,1169,NaN,2800000,19 mars 2013,13,75676,norrmalm,37,2,3
10472,olofsgatan 18,917,NaN,2360000,28 februari 2013,8,73750,norrmalm,32,1,3
10473,regeringsgatan 70 d,3231,NaN,3600000,17 februari 2013,3,52174,norrmalm,69,3,None
10474,drottninggatan 80,2647,NaN,3250000,22 januari 2013,5,73864,norrmalm,44,2,3


# Yelp API calls

In [2]:
yelp_df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/data/UNFINISHED_yelp_df_CLEANED.csv")


In [25]:
yelp_api = YelpAPI("KulP_1xAbhj4PcwcltixYR5hz4qMJ2aarTp4uNP_bBED4CsgP1nqY0bZrDxRMMSsZYwqvirOQ1Dy--6v3Y2yS4lBVPmfebDVdXGukr74OZEKRNoivTBiORBJ0v_iYXYx")

#A 'suggested search area' of 500 is used, although, the docs tell us the following: 
#This field is used as a suggestion to the search. The actual search radius may be lower 
#than the suggested radius in dense urban areas, and higher in regions of less business density
#hopefully, it does give some quantifiable value as to the 'closeness' to points of interest a apartment has

k = 9990
def get_POI(adress):
    input_adress = adress
    response = yelp_api.search_query(location=input_adress, radius=500, limit=1)
    return response

def get_yelp_values(df, k):
    try:
        for i in range(k, len(df)):
            response = get_POI(df["adress"][i])
            print("Getting values from adress:", df["adress"][i], "from row: " , i)
            
            lat = ( response['region']['center']['latitude'] )
            long = ( response['region']['center']['longitude'] )
            POI = ( response['total'])
            df.at[i, 'Latitude'] = lat
            df.at[i, 'Longitude'] = long
            df.at[i, 'NearbyPOIs'] = POI
            
        return df
    except Exception as e:
        print(e)
        print("\n","error")
        k = i+1
        
        error_handler(df,k)
        
        
def error_handler(df,k):
    get_yelp_values(df,k)
    return 

get_yelp_values(yelp_df,k)

#DONE UNTIL 9989, begin at k = 9990

yelp_df.to_csv("UNFINISHED_yelp_df_CLEANED.csv",index=False)

LOCATION_NOT_FOUND: Could not execute search, try specifying a more exact location.

 error
Getting values from adress: vintervägen 26 from row:  7827
Getting values from adress: rådjursstigen 20 from row:  7828
Getting values from adress: solnavägen 43 from row:  7829
Getting values from adress: wiboms väg 25 from row:  7830
Getting values from adress: björnstigen 125 from row:  7831
Getting values from adress: kyrkbacken 9 from row:  7832
Getting values from adress: gustafsvägen 3 from row:  7833
Getting values from adress: garvis carlssons gata 20 from row:  7834
Getting values from adress: solnavägen 39 from row:  7835
Getting values from adress: näckrosvägen 7 from row:  7836
Getting values from adress: kolonnvägen 68 from row:  7837
Getting values from adress: råsundavägen 133b from row:  7838
Getting values from adress: slåttervägen 6 from row:  7839
Getting values from adress: olof af acrels väg 5 from row:  7840
Getting values from adress: råsundavägen 74 from row:  7841
Getti

Getting values from adress: erik sandbergs gata 6 from row:  7957
Getting values from adress: östervägen 11 from row:  7958
Getting values from adress: solnavägen 39 from row:  7959
Getting values from adress: vintervägen 26 from row:  7960
Getting values from adress: johan enbergs väg 28 4tr. from row:  7961
Getting values from adress: jungfrudansen 13 from row:  7962
Getting values from adress: evenemangsgatan 30 from row:  7963
Getting values from adress: arvid tydéns allé 2 from row:  7964
Getting values from adress: södra långgatan 41 from row:  7965
Getting values from adress: gustav iii:s boulevard 141 from row:  7966
Getting values from adress: rosstigen 9 from row:  7967
LOCATION_NOT_FOUND: Could not execute search, try specifying a more exact location.

 error
Getting values from adress: bygatan 13 from row:  7969
Getting values from adress: kyrkbacken 9 from row:  7970
Getting values from adress: plogvägen 5 from row:  7971
Getting values from adress: huvudstagatan 1b from r

Getting values from adress: vasavägen 29 from row:  8091
Getting values from adress: åkersvägen 8 from row:  8092
Getting values from adress: ripstigen 8 from row:  8093
Getting values from adress: uppfartsvägen 13 from row:  8094
Getting values from adress: s:t ansgars väg 10 a from row:  8095
Getting values from adress: fridensborgsvägen 137 from row:  8096
Getting values from adress: kolonnvägen 72 from row:  8097
Getting values from adress: nämndemansgatan 25 from row:  8098
Getting values from adress: mittfältsgatan 9 from row:  8099
Getting values from adress: förrådsgatan 4 from row:  8100
Getting values from adress: fridensborgsvägen 58 from row:  8101
Getting values from adress: jonstorpsvägen 16 from row:  8102
Getting values from adress: järva skjutbaneväg 35 from row:  8103
Getting values from adress: brotorpsvägen 4 from row:  8104
Getting values from adress: årebacken 12 from row:  8105
Getting values from adress: edvin adolphsons väg 6 from row:  8106
Getting values from

Getting values from adress: arvid tydéns allé 22 from row:  8221
Getting values from adress: sylvangatan 5 from row:  8222
Getting values from adress: virebergsvägen 15 from row:  8223
Getting values from adress: evenemangsgatan 40 from row:  8224
Getting values from adress: tottvägen 4a from row:  8225
Getting values from adress: släggbacken 6 from row:  8226
Getting values from adress: mittfältsgatan 3 from row:  8227
Getting values from adress: huvudstagatan 29 from row:  8228
Getting values from adress: jungfrudansen 52 from row:  8229
Getting values from adress: garvis carlssons gata 6 from row:  8230
Getting values from adress: garvis carlssons gata 6 from row:  8231
Getting values from adress: garvis carlssons gata 8 from row:  8232
Getting values from adress: fridensborgsvägen 121 from row:  8233
Getting values from adress: gustav iii:s boulevard 139 from row:  8234
Getting values from adress: näckrosvägen 5 from row:  8235
Getting values from adress: järva skjutbaneväg 33 from

Getting values from adress: evenemangsgatan 24 from row:  8354
Getting values from adress: galoppvägen 14 from row:  8355
Getting values from adress: armégatan 11 - vån 2 from row:  8356
Getting values from adress: armégatan 11a from row:  8357
Getting values from adress: jungfrudansen 11 from row:  8358
Getting values from adress: frösundaviks allé 14 from row:  8359
Getting values from adress: evenemangsgatan 12 from row:  8360
Getting values from adress: kolonnvägen 72 from row:  8361
Getting values from adress: näckrosvägen 18 from row:  8362
Getting values from adress: ängkärrsgatan 3 from row:  8363
Getting values from adress: algatan 13 from row:  8364
Getting values from adress: östervägen 11 from row:  8365
Getting values from adress: förrådsgatan 2 from row:  8366
Getting values from adress: kolonnvägen 68 from row:  8367
Getting values from adress: kolonnvägen 68 våning 3 from row:  8368
Getting values from adress: uppfartsvägen 11 from row:  8369
Getting values from adress:

...

Getting values from adress: kammakargatan 33 a from row:  9680
Getting values from adress: smala gränd 3 from row:  9681
Getting values from adress: kammakargatan 44a from row:  9682
Getting values from adress: jutas backe 5 from row:  9683
Getting values from adress: herkulesgatan 24 from row:  9684
Getting values from adress: tegnérgatan 55d from row:  9685
Getting values from adress: david bagares gata 22 from row:  9686
Getting values from adress: blekholmsterrassen 7 from row:  9687
Getting values from adress: olof palmes gata 12 from row:  9688
Getting values from adress: tegnérgatan 55a from row:  9689
Getting values from adress: torsplan 8 from row:  9690
Getting values from adress: luntmakargatan 12 from row:  9691
Getting values from adress: apelbergsgatan 60 from row:  9692
Getting values from adress: tegnérgatan 55b from row:  9693
Getting values from adress: döbelnsgatan 13 from row:  9694
Getting values from adress: wallingatan 13 from row:  9695
Getting values from adres

Getting values from adress: kammakargatan 21 from row:  9813
Getting values from adress: holländargatan 9 a from row:  9814
Getting values from adress: snickarbacken 7 from row:  9815
Getting values from adress: tegnérgatan 55 d from row:  9816
Getting values from adress: adolf fredriks kyrkogata 3 from row:  9817
Getting values from adress: klara östra kyrkogata 8f from row:  9818
Getting values from adress: luntmakargatan 12 from row:  9819
Getting values from adress: david bagares gata 20 b from row:  9820
Getting values from adress: wallingatan 31 from row:  9821
Getting values from adress: kammakargatan 70 from row:  9822
Getting values from adress: kammakargatan 27 from row:  9823
Getting values from adress: kammakargatan 35 from row:  9824
Getting values from adress: david bagares gata 10 from row:  9825
Getting values from adress: drottninggatan 90b from row:  9826
Getting values from adress: snickarbacken 7 from row:  9827
Getting values from adress: david bagares gata 26c fro

Getting values from adress: regeringsgatan 70 a from row:  9946
Getting values from adress: drottninggatan 90 b from row:  9947
Getting values from adress: dalagatan 6 a from row:  9948
Getting values from adress: kammakargatan 70 from row:  9949
Getting values from adress: herkulesgatan 24 from row:  9950
Getting values from adress: herkulesgatan 26 from row:  9951
Getting values from adress: kammakargatan 70 from row:  9952
Getting values from adress: luntmakargatan 36 from row:  9953
Getting values from adress: tegnérgatan 19 from row:  9954
Getting values from adress: adolf fredriks kyrkogata 3 from row:  9955
Getting values from adress: regeringsgatan 70 a from row:  9956
Getting values from adress: johannesgatan 10 from row:  9957
Getting values from adress: luntmakargatan 10 from row:  9958
Getting values from adress: olof palmes gata 12 from row:  9959
Getting values from adress: kammakargatan 64 b from row:  9960
Getting values from adress: blekholmsterrassen 9 from row:  9961

ACCESS_LIMIT_REACHED: You've reached the access limit for this client. See instructions for requesting a higher access limit at https://www.yelp.com/developers/documentation/v3/rate_limiting

 error
ACCESS_LIMIT_REACHED: You've reached the access limit for this client. See instructions for requesting a higher access limit at https://www.yelp.com/developers/documentation/v3/rate_limiting

 error
ACCESS_LIMIT_REACHED: You've reached the access limit for this client. See instructions for requesting a higher access limit at https://www.yelp.com/developers/documentation/v3/rate_limiting

 error
ACCESS_LIMIT_REACHED: You've reached the access limit for this client. See instructions for requesting a higher access limit at https://www.yelp.com/developers/documentation/v3/rate_limiting

 error
ACCESS_LIMIT_REACHED: You've reached the access limit for this client. See instructions for requesting a higher access limit at https://www.yelp.com/developers/documentation/v3/rate_limiting

 error
ACCES

KeyboardInterrupt: 

In [29]:
yelp_df[yelp_df["NearbyPOIs"].isnull()]

,adress,fee,features,sale_price,sold_date,value_dev,ppsqm,district,size,rooms,floor,Latitude,Longitude,NearbyPOIs
1826,norra stationsgatan 53,1867,2.0,3020000,15 februari 2021,12,104138,vasastan,29,1,NaN,NaN,NaN,NaN
7733,parkvägen 2,4415,2.0,6500000,6 januari 2022,±0,84416,solna,77,3,NaN,NaN,NaN,NaN
7737,parkvägen 2,2295,3.0,3900000,4 januari 2022,8,97500,solna,40,2,NaN,NaN,NaN,NaN
7763,tvärvägen 3,3494,2.0,3900000,17 december 2021,±0,67241,solna,58,2,2.0,NaN,NaN,NaN
7793,parkvägen 2,2008,3.0,3050000,8 december 2021,±0,87143,solna,35,2,12.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10471,kammakargatan 70,1169,NaN,2800000,19 mars 2013,13,75676,norrmalm,37,2,3.0,NaN,NaN,NaN
10472,olofsgatan 18,917,NaN,2360000,28 februari 2013,8,73750,norrmalm,32,1,3.0,NaN,NaN,NaN
10473,regeringsgatan 70 d,3231,NaN,3600000,17 februari 2013,3,52174,norrmalm,69,3,NaN,NaN,NaN,NaN
10474,drottninggatan 80,2647,NaN,3250000,22 januari 2013,5,73864,norrmalm,44,2,3.0,NaN,NaN,NaN


In [31]:
yelp_df = pd.read_csv("C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/notebooks/UNFINISHED_yelp_df_CLEANED.csv")


In [33]:
yelp_df[yelp_df["NearbyPOIs"].isnull()]

,adress,fee,features,sale_price,sold_date,value_dev,ppsqm,district,size,rooms,floor,Latitude,Longitude,NearbyPOIs
1826,norra stationsgatan 53,1867,2.0,3020000,15 februari 2021,12,104138,vasastan,29,1,NaN,NaN,NaN,NaN
7733,parkvägen 2,4415,2.0,6500000,6 januari 2022,±0,84416,solna,77,3,NaN,NaN,NaN,NaN
7737,parkvägen 2,2295,3.0,3900000,4 januari 2022,8,97500,solna,40,2,NaN,NaN,NaN,NaN
7763,tvärvägen 3,3494,2.0,3900000,17 december 2021,±0,67241,solna,58,2,2.0,NaN,NaN,NaN
7793,parkvägen 2,2008,3.0,3050000,8 december 2021,±0,87143,solna,35,2,12.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10471,kammakargatan 70,1169,NaN,2800000,19 mars 2013,13,75676,norrmalm,37,2,3.0,NaN,NaN,NaN
10472,olofsgatan 18,917,NaN,2360000,28 februari 2013,8,73750,norrmalm,32,1,3.0,NaN,NaN,NaN
10473,regeringsgatan 70 d,3231,NaN,3600000,17 februari 2013,3,52174,norrmalm,69,3,NaN,NaN,NaN,NaN
10474,drottninggatan 80,2647,NaN,3250000,22 januari 2013,5,73864,norrmalm,44,2,3.0,NaN,NaN,NaN
